In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE180395"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE180395"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE180395.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE180395.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE180395.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import re
import gzip
from typing import Optional, Dict, Any, Callable

# Function to extract sample characteristics from GEO series matrix file
def extract_sample_info(file_path):
    characteristics_dict = {}
    background_info = {}
    
    with gzip.open(file_path, 'rt') as file:
        line_count = 0
        for line in file:
            line = line.strip()
            line_count += 1
            
            # Extract background information
            if line.startswith('!Series_'):
                key = line.split('\t')[0]
                value = line.split('\t')[1] if len(line.split('\t')) > 1 else ""
                background_info[key] = value
            
            # Extract sample characteristics
            if line.startswith('!Sample_characteristics_ch'):
                parts = line.split('\t')
                key_idx = len(characteristics_dict)
                values = [v.strip('"') for v in parts[1:]]
                unique_values = list(set([v for v in values if v and v != "NA"]))
                characteristics_dict[key_idx] = unique_values
            
            # Limit processing to avoid memory issues
            if line_count > 5000:
                break
    
    return background_info, characteristics_dict

# Process the GEO matrix file
file_path = os.path.join(in_cohort_dir, "GSE180395_series_matrix.txt.gz")

# Check if file exists
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
    is_gene_available = False
    is_trait_available = False
else:
    # Extract information
    background_info, characteristics_dict = extract_sample_info(file_path)
    
    # Print extracted info for debugging
    print("Background Information:")
    for key, value in background_info.items():
        print(f"{key}\t{value}")
    
    print("\nSample Characteristics Dictionary:")
    print(characteristics_dict)
    
    # 1. Gene Expression Data Availability
    # Based on the series title and summary, this appears to be a transcriptome study
    is_gene_available = True
    
    # 2. Variable Availability
    # From the output of the previous step, trait information is in row 0
    trait_row = 0  # 'sample group' contains disease vs control information
    age_row = None  # No age information available in the provided characteristics
    gender_row = None  # No gender information available in the provided characteristics
    
    # Check trait data availability
    is_trait_available = trait_row is not None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """Convert disease status to binary: 1 for disease, 0 for control/living donor."""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    match = re.search(r':\s*(.*)', value)
    if match:
        value = match.group(1).strip()
    else:
        value = value.strip()
    
    # Living donor is considered as control
    if "Living donor" in value:
        return 0
    # All other values indicate some form of disease/condition
    else:
        return 1

def convert_age(value: str) -> Optional[float]:
    """Convert age to float."""
    # Function defined but not used as age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary: 0 for female, 1 for male."""
    # Function defined but not used as gender data is not available
    return None

# 3. Save Metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if is_trait_available:
    # Read the clinical data
    clinical_data = pd.read_csv(file_path, sep='\t', comment='!', compression='gzip')
    
    # Use the library function to extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_data, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical data
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Review the gene identifiers from the output above
# The identifiers appear to be probe IDs from a microarray, as they have 
# a specific format with numbers followed by "_at"
# These are not standard human gene symbols and will need to be mapped

# Based on biomedical knowledge, these are likely Affymetrix probe IDs
# which need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 6: Gene Identifier Mapping

In [ ]:
# Get the file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data again
gene_data = get_genetic_data(matrix_file_path)

# Check whether gene expression data has any rows
if gene_data.shape[0] == 0:
    print("Warning: Gene expression data is empty.")
    is_gene_available = False
else:
    # 1. Compare the probe ID formats in both datasets
    print("Sample probe IDs in gene expression data:")
    print(gene_data.index[:5])
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    print("\nSample probe IDs in gene annotation:")
    print(gene_annotation['ID'].head())
    
    # 2. Get the gene mapping dataframe
    prob_col = 'ID'  # This contains the probe IDs like '10000_at'
    gene_col = 'ENTREZ_GENE_ID'  # This contains the Entrez Gene IDs
    
    # Create mapping dataframe
    gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
    
    # Check the overlap between gene expression probe IDs and mapping probe IDs
    expression_probes = set(gene_data.index)
    mapping_probes = set(gene_mapping['ID'])
    overlap = expression_probes.intersection(mapping_probes)
    
    print(f"\nOverlap between expression probes and mapping probes: {len(overlap)} out of {len(expression_probes)} expression probes")
    
    # 3. Modify the probe IDs in the mapping to match the expression data if needed
    if len(overlap) == 0:
        # Try to match by removing the "_at" suffix if present
        # Check if we need to add or remove suffix
        sample_expr_id = list(expression_probes)[0]
        sample_map_id = list(mapping_probes)[0]
        
        print(f"Sample expression probe ID: {sample_expr_id}")
        print(f"Sample mapping probe ID: {sample_map_id}")
        
        # Convert Entrez IDs to appropriate format for mapping
        # Since our expression data has format like "10000_at", ensure mapping IDs match this format
        if "_at" in sample_expr_id and "_at" not in sample_map_id:
            print("Adding '_at' suffix to mapping probe IDs...")
            gene_mapping['ID'] = gene_mapping['ID'] + "_at"
        elif "_at" not in sample_expr_id and "_at" in sample_map_id:
            print("Removing '_at' suffix from mapping probe IDs...")
            gene_mapping['ID'] = gene_mapping['ID'].str.replace("_at", "")
        
        # Check overlap again after modification
        mapping_probes = set(gene_mapping['ID'])
        overlap = expression_probes.intersection(mapping_probes)
        print(f"After adjustment, overlap: {len(overlap)} out of {len(expression_probes)} expression probes")
    
    # 3. Apply the gene mapping to convert probe-level data to gene-level data
    if len(overlap) > 0:
        # This converts expression values from probes to genes 
        gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)
        
        # Let's see how many genes we have after mapping
        print(f"\nGene expression data after mapping:")
        print(f"Shape: {gene_data_mapped.shape}")
        if gene_data_mapped.shape[0] > 0:
            print("First 5 genes:")
            print(gene_data_mapped.index[:5])
            
            # Update gene_data with the mapped data
            gene_data = gene_data_mapped
            
            # Save the processed gene data
            os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
            gene_data.to_csv(out_gene_data_file)
            print(f"Gene expression data saved to {out_gene_data_file}")
        else:
            print("ERROR: Gene mapping resulted in empty dataframe.")
            is_gene_available = False
    else:
        print("ERROR: No overlap between gene expression probes and mapping probes.")
        print("Cannot proceed with gene mapping.")
        is_gene_available = False

# Update metadata if mapping failed
if not is_gene_available:
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 7: Data Normalization and Linking

In [ ]:
# 1. First let's check the situation with our gene data
try:
    # Get the file paths
    soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)
    
    # We know from previous steps that the gene mapping resulted in an empty dataframe
    # Let's extract the genetic data again
    gene_data = get_genetic_data(matrix_file_path)
    
    # Check if the gene data extraction worked
    print(f"Original gene expression data shape: {gene_data.shape}")
    
    # Due to issues with gene mapping in previous steps, let's use the original probe-level data
    if gene_data.shape[0] > 0:
        print("Using original probe-level data instead of mapped gene data")
        # Set index name to "Gene" to maintain expected format
        gene_data.index.name = 'Gene'
        
        # Save the gene data directly
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        gene_data.to_csv(out_gene_data_file)
        print(f"Probe-level data saved to {out_gene_data_file}")
    else:
        print("ERROR: Gene data extraction failed")
        is_gene_available = False
except Exception as e:
    print(f"Error with gene data processing: {e}")
    is_gene_available = False

# 2. Extract and process clinical data from raw file
try:
    # Re-load the sample characteristics
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    _, clinical_raw = get_background_and_clinical_data(matrix_file_path, background_prefixes, clinical_prefixes)
    
    # Function to convert trait values based on sample description
    def convert_trait(value):
        """Convert sample groups to binary trait values"""
        if value is None or not isinstance(value, str):
            return None
            
        # Extract the value after the colon if present
        match = re.search(r'sample group:\s*(.*)', value)
        if match:
            value = match.group(1).strip()
        else:
            value = value.strip()
        
        # Living donor is considered as control
        if "Living donor" in value:
            return 0
        # All other values indicate some form of disease/condition
        elif any(x in value for x in ["GN", "LN", "nephritis", "FSGS", "DN", "amyloidosis", "MN", "AKI"]):
            return 1
        else:
            return None
    
    # Create a binary trait based on sample groups
    trait_row = 0  # From inspection of the clinical_raw data
    
    # Process clinical features and extract trait information
    if trait_row is not None:
        clinical_df = clinical_raw.copy()
        clinical_features = geo_select_clinical_features(
            clinical_df, 
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=None,  # No age information available
            convert_age=None,
            gender_row=None,  # No gender information available
            convert_gender=None
        )
        
        # Transpose to get samples as rows
        clinical_features = clinical_features.T
        
        # Save clinical features
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        print(f"Clinical features shape: {clinical_features.shape}")
    else:
        print("No trait information available in clinical data")
        is_trait_available = False
        clinical_features = pd.DataFrame()
except Exception as e:
    print(f"Error processing clinical data: {e}")
    is_trait_available = False
    clinical_features = pd.DataFrame()

# 3. Link clinical and gene data if both are available
if is_gene_available and is_trait_available and gene_data.shape[0] > 0 and clinical_features.shape[0] > 0:
    try:
        # Ensure gene data is formatted with genes as rows and samples as columns
        gene_data.index.name = 'Gene'
        
        # Make sample IDs match between datasets
        # In gene_data, the columns contain GSM IDs
        # In clinical_features, the rows contain GSM IDs
        common_samples = list(set(clinical_features.index) & set(gene_data.columns))
        print(f"Number of common samples between datasets: {len(common_samples)}")
        
        if len(common_samples) == 0:
            print("WARNING: No matching sample IDs between clinical and genetic data.")
            is_gene_available = False
        else:
            # Filter both datasets to include only common samples
            clinical_subset = clinical_features.loc[common_samples]
            gene_subset = gene_data[common_samples]
            
            # Transpose gene data to have samples as rows
            gene_subset_t = gene_subset.T
            
            # Link the datasets
            linked_data = pd.concat([clinical_subset, gene_subset_t], axis=1)
            print(f"Linked data shape: {linked_data.shape}")
            
            # 4. Handle missing values in the linked data
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Data shape after handling missing values: {linked_data.shape}")
            
            # 5. Determine if trait is biased
            is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    except Exception as e:
        print(f"Error linking data: {e}")
        is_trait_biased = True
        linked_data = pd.DataFrame()
else:
    print("Cannot link data: gene or trait data unavailable")
    is_trait_biased = True
    linked_data = pd.DataFrame()

# 6. Make final determination about data usability
note = "Dataset contains kidney disease gene expression data. Processing encountered issues with gene ID mapping."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available and gene_data.shape[0] > 0,
    is_trait_available=is_trait_available and clinical_features.shape[0] > 0,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save linked data if usable
if is_usable and linked_data.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data not saved due to quality issues")